In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from hmmlearn.hmm import GaussianHMM
from sklearn.metrics import f1_score

In [2]:
# Use the result from previous notebook
train = pd.read_csv('../data/train_2.csv')
test = pd.read_csv('../data/test_2.csv')

In [3]:
BATCHES = np.array([0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 65, 70])
CATEGORIES = np.array([1, 1, 2, 3, 5, 4, 2, 3, 4, 5, 6, 3, 4, 6, 2, 5, 4, 5, 6, 3, 6, 6])

# I would not advice CATEGORY > 3 with hmmlearn... (will take a LONG time)
CATEGORY = 3

signal = np.concatenate((train['signal'].values, test['signal'].values))

ix = np.where(CATEGORIES == CATEGORY)[0]
starts = BATCHES[ix]
ends = BATCHES[ix + 1]

X = []
y = []
for start, end in zip(starts, ends):
    subsignal = signal[start*100_000:end*100_000]
    if start < 50:
        subchannels = train['open_channels'].values[start*100_000:end*100_000]
    else:
        subchannels = [-1]*((end-start)*100_000)
        
    if start == 35:
        subsignal = list(subsignal[:100000]) + list(subsignal[-100000:])
        subchannels = list(subchannels[:100000]) + list(subchannels[-100000:])
    
    X.extend(subsignal)
    y.extend(subchannels)
    
X = np.array(X)
y = np.array(y)
print(len(X), len(y))

900000 900000


In [4]:
# Estimate the transition matrix based on the ground truth
Ptran = np.array([[0     , 0.0067, 0     , 0     ],
                  [0.0373, 0     , 0.2762, 0.0230],
                  [0     , 0.1991, 0     , 0     ],
                  [0     , 0.0050, 0     , 0     ]])
States = [1, 1, 0, 0]

def calculate_matrix(transition_matrix, states, number_processes):
    """
    Expand a transition matrix to model separate processes.
    If max(open_channels) = K, then we assume K 0/1 processes. 
    E.g. our data category 3 corresponds to a maximum
    of 3 open_channels, so 3 processes.
    
    We create model a combination_with_repetition(3, 4) = 20
    transition matrix. The first row & col corresponds to all
    processes being in the first hidden state (1, 1, 1). The
    second row & col corresponds to (1, 1, 2), and so on until
    (4, 4, 4).
    
    To calculate the transition probability from (1, 2, 2) to
    (1, 1, 3), we calculate P(1->1) * P(2->1) * P(2->3). But
    also for all permutations (e.g. (2, 1, 2) and (3, 1, 1)).
    In the end, we normalize our transition matrix.
    """
    # Fill in diagonals such that each row sums to 1
    for i in range(transition_matrix.shape[0]):
        transition_matrix[i, i] = 1 - np.sum(transition_matrix[i, :])

    n0 = len(states)
    new_transition_matrix = transition_matrix.copy()
    new_states = [(x,) for x in range(n0)]
    for process in range(1, number_processes):
        # We expand our current transition matrix (that models up to `process` number
        # of separate processes) its' dimensions by n0. We basically add another
        # possible state transition for a new process.
        nc = new_transition_matrix.shape[0]
        temp_transition_matrix = np.zeros((n0*nc, n0*nc))
        temp_states = []
        for i in range(n0):
            temp_states.extend([s + (i,) for s in new_states])
            for j in range(n0):
                # We add i -> j as our final transition
                temp_transition_matrix[i*nc:(i+1)*nc, j*nc:(j+1)*nc] = transition_matrix[i][j] * new_transition_matrix
              
        # We now group similar processes together to reduce our matrix. 
        # E.g. (1, 2, 3) is the same as (2, 3, 1)
        new_states = sorted(list(set([tuple(sorted(x)) for x in temp_states])))
        new_transition_matrix = np.zeros((len(new_states), len(new_states)))
        for i in range(len(new_states)):
            ix_i = [k for k, x in enumerate(temp_states) if tuple(sorted(x)) == new_states[i]]
            for j in range(len(new_states)):
                ix_j = [k for k, x in enumerate(temp_states) if tuple(sorted(x)) == new_states[j]]
                new_transition_matrix[i, j] = np.sum(temp_transition_matrix[ix_i, :][:, ix_j])
                new_transition_matrix[i, j] /= len(ix_i)
    
    new_channels = []
    for s in new_states:
        new_channels.append(sum([states[x] for x in s]))
    new_channels= np.array(new_channels)
        
    return new_transition_matrix, new_channels

Ptran, States = calculate_matrix(Ptran, States, 3)
print('Transition matrix shape: {}'.format(Ptran.shape))

# Estimate means and covs per unique ground truth value
means = []
covs = []
for c in sorted(np.unique(y[y >= 0])):
    means.append(np.mean(X[y == c]))
    covs.append(np.cov(X[y == c]))
    
means = [means[c] for c in States]
covs = [covs[c] for c in States]
    
# Defining our HMM
hmm = GaussianHMM(
    n_components=len(States),           # Number of hidden states
    n_iter=50,                          # Total number of iterations
    verbose=True,                       # Show logs
    algorithm='map',                    # Use maximum a posteriori instead of Viterbi
    params='stmc',                      # Optimize start probs, transmat, means, covs
    random_state=42,
    init_params='s',                    # Manually initialize all but start probabilities
    covariance_type='full',             # Separate covariance per hidden state
    tol=1                               # Convergence criterion (set high for fast execution)
)

# Initialize the parameters of our HMM
hmm.n_features = 1
hmm.means_ = np.array(means).reshape(-1 ,1)
hmm.covars_ = np.array(covs).reshape(-1, 1, 1)
hmm.transmat_ = Ptran

# Fit our HMM (this takes a while)
_ = hmm.fit(X.reshape(-1, 1), lengths=[100000]*(len(X) // 100000))

Transition matrix shape: (20, 20)


         1     -501940.5201             +nan
         2     -490739.9518      +11200.5683
         3     -490293.7547        +446.1971
         4     -490141.4750        +152.2797
         5     -490070.0501         +71.4249
         6     -490031.7036         +38.3465
         7     -490009.3388         +22.3648
         8     -489995.4640         +13.8748
         9     -489986.3995          +9.0645
        10     -489980.1955          +6.2039
        11     -489975.7609          +4.4346
        12     -489972.4581          +3.3028
        13     -489969.9012          +2.5569
        14     -489967.8491          +2.0520
        15     -489966.1469          +1.7022
        16     -489964.6919          +1.4550
        17     -489963.4147          +1.2772
        18     -489962.2683          +1.1463
        19     -489961.2228          +1.0455
        20     -489960.2611          +0.9617


In [5]:
# Make predictions
preds = np.array(States)[hmm.predict(X.reshape(-1, 1), lengths=[100000]*(len(X) // 100000))]

# Our naive HMM
print(f1_score(y[y >= 0], preds[y >= 0], average='macro'))

0.9866748988341756
